# Requirements

pandas

numpy

matplotlib

seaborn

Scikit-Learn 

polars

H2O.ai

In [1]:
import polars as pl
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pl.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pl.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [3]:
train.head(2)

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str,bool
"""0001_01""","""Europa""",false,"""B/0/P""","""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,"""Maham Ofracculy""",false
"""0002_01""","""Earth""",false,"""F/0/S""","""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,"""Juanna Vines""",true


In [4]:
test.head(2)

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str
"""0013_01""","""Earth""",true,"""G/3/S""","""TRAPPIST-1e""",27.0,false,0.0,0.0,0.0,0.0,0.0,"""Nelly Carsoning"""
"""0018_01""","""Earth""",false,"""F/4/S""","""TRAPPIST-1e""",19.0,false,0.0,9.0,0.0,2823.0,0.0,"""Lerome Peckers"""


# Basic Statistics

In [5]:
train.describe()

statistic,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64
"""count""","""8693""","""8492""",8476.0,"""8494""","""8511""",8514.0,8490.0,8512.0,8510.0,8485.0,8510.0,8505.0,"""8493""",8693.0
"""null_count""","""0""","""201""",217.0,"""199""","""182""",179.0,203.0,181.0,183.0,208.0,183.0,188.0,"""200""",0.0
"""mean""",null,null,0.358306,null,null,28.82793,0.023439,224.687617,458.077203,173.729169,311.138778,304.854791,null,0.503624
"""std""",null,null,null,null,null,14.489021,null,666.717663,1611.48924,604.696458,1136.705535,1145.717189,null,null
"""min""","""0001_01""","""Earth""",0.0,"""A/0/P""","""55 Cancri e""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Aard Curle""",0.0
"""25%""",null,null,null,null,null,19.0,null,0.0,0.0,0.0,0.0,0.0,null,null
"""50%""",null,null,null,null,null,27.0,null,0.0,0.0,0.0,0.0,0.0,null,null
"""75%""",null,null,null,null,null,38.0,null,47.0,76.0,27.0,59.0,46.0,null,null
"""max""","""9280_02""","""Mars""",1.0,"""T/3/P""","""TRAPPIST-1e""",79.0,1.0,14327.0,29813.0,23492.0,22408.0,24133.0,"""Zubeneb Pasharne""",1.0


In [6]:
train.shape

(8693, 14)

In [7]:
train.null_count()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,201,217,199,182,179,203,181,183,208,183,188,200,0


In [8]:
test.shape

(4277, 13)

In [9]:
test.null_count()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,87,93,100,92,91,93,82,106,98,101,80,94


In [10]:
# Null Percentage

In [11]:
# train
null_percentage_train = train.null_count() / train.height * 100
null_percentage_train

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,2.312205,2.496261,2.289198,2.093639,2.059128,2.335212,2.082135,2.105142,2.39273,2.105142,2.16266,2.300702,0.0


In [12]:
# test
null_percentage_test = test.null_count() / test.height * 100
null_percentage_test

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,2.034136,2.174421,2.338087,2.15104,2.12766,2.174421,1.917232,2.478373,2.291326,2.361468,1.87047,2.197802


In [13]:
# Get unique value counts for each column
unique_value_counts = {col: train[col].unique().shape[0] for col in train.columns}

# Display the unique value counts
for col, count in unique_value_counts.items():
    print(f"Column '{col}': Unique Value Count -> {count}")

Column 'PassengerId': Unique Value Count -> 8693
Column 'HomePlanet': Unique Value Count -> 4
Column 'CryoSleep': Unique Value Count -> 3
Column 'Cabin': Unique Value Count -> 6561
Column 'Destination': Unique Value Count -> 4
Column 'Age': Unique Value Count -> 81
Column 'VIP': Unique Value Count -> 3
Column 'RoomService': Unique Value Count -> 1274
Column 'FoodCourt': Unique Value Count -> 1508
Column 'ShoppingMall': Unique Value Count -> 1116
Column 'Spa': Unique Value Count -> 1328
Column 'VRDeck': Unique Value Count -> 1307
Column 'Name': Unique Value Count -> 8474
Column 'Transported': Unique Value Count -> 2


In [14]:
# Removing unnecessary columns

In [15]:
train = train.drop(['PassengerId','Cabin','Name'])

In [16]:
test = test.drop(['PassengerId','Cabin'])

In [17]:
test = test.drop('Name')

# Handling Nan Vals.

In [18]:
train.describe()

statistic,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""8492""",8476.0,"""8511""",8514.0,8490.0,8512.0,8510.0,8485.0,8510.0,8505.0,8693.0
"""null_count""","""201""",217.0,"""182""",179.0,203.0,181.0,183.0,208.0,183.0,188.0,0.0
"""mean""",null,0.358306,null,28.82793,0.023439,224.687617,458.077203,173.729169,311.138778,304.854791,0.503624
"""std""",null,null,null,14.489021,null,666.717663,1611.48924,604.696458,1136.705535,1145.717189,null
"""min""","""Earth""",0.0,"""55 Cancri e""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,null,19.0,null,0.0,0.0,0.0,0.0,0.0,null
"""50%""",null,null,null,27.0,null,0.0,0.0,0.0,0.0,0.0,null
"""75%""",null,null,null,38.0,null,47.0,76.0,27.0,59.0,46.0,null
"""max""","""Mars""",1.0,"""TRAPPIST-1e""",79.0,1.0,14327.0,29813.0,23492.0,22408.0,24133.0,1.0


In [19]:
# Identify numerical and categorical columns
num_cols = [col for col, dtype in zip(train.columns, train.dtypes) if dtype in [pl.Int64, pl.Float64, pl.UInt64]]
cat_cols = [col for col, dtype in zip(train.columns, train.dtypes) if dtype in [pl.Utf8, pl.Categorical]]

# Display the results
print("Numerical Columns:", num_cols)
print("Categorical Columns:", cat_cols)

Numerical Columns: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
Categorical Columns: ['HomePlanet', 'Destination']


In [20]:
num_cols.append('CryoSleep')

In [21]:
num_cols.append('VIP')

In [22]:
num_cols

['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'CryoSleep',
 'VIP']

In [23]:
cat_cols

['HomePlanet', 'Destination']

In [24]:
%time

# 1. Impute median value for numerical columns
for col in num_cols:
    median_value = train[col].median()
    train = train.with_columns(
        pl.when(pl.col(col).is_null())
        .then(median_value)
        .otherwise(pl.col(col))
        .alias(col)
    )

# 2. Encode categorical columns to numerical values
cat_df = train.select(cat_cols).to_pandas()
label_encoders = {}

for col in cat_cols:
    encoder = LabelEncoder()
    cat_df[col] = cat_df[col].fillna("missing")  # Temporarily fill NaN with placeholder
    cat_df[col] = encoder.fit_transform(cat_df[col])
    label_encoders[col] = encoder

# Replace placeholder with NaN for imputation
cat_df.replace({"missing": np.nan}, inplace=True)

# Apply KNN Imputer
knn_imputer = KNNImputer(n_neighbors=3, weights="uniform", metric="nan_euclidean")
cat_imputed = knn_imputer.fit_transform(cat_df)

# Convert imputed values back to original categorical form
for i, col in enumerate(cat_cols):
    cat_df[col] = label_encoders[col].inverse_transform(cat_imputed[:, i].round().astype(int))

# Convert the imputed data back to a Polars DataFrame
cat_imputed_df = pl.DataFrame(cat_df)

# Update the original Polars DataFrame with imputed categorical values
train = train.with_columns(cat_imputed_df)

print(train)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs
shape: (8_693, 11)
┌────────────┬───────────┬───────────────┬──────┬───┬──────────────┬────────┬────────┬─────────────┐
│ HomePlanet ┆ CryoSleep ┆ Destination   ┆ Age  ┆ … ┆ ShoppingMall ┆ Spa    ┆ VRDeck ┆ Transported │
│ ---        ┆ ---       ┆ ---           ┆ ---  ┆   ┆ ---          ┆ ---    ┆ ---    ┆ ---         │
│ str        ┆ f64       ┆ str           ┆ f64  ┆   ┆ f64          ┆ f64    ┆ f64    ┆ bool        │
╞════════════╪═══════════╪═══════════════╪══════╪═══╪══════════════╪════════╪════════╪═════════════╡
│ Europa     ┆ 0.0       ┆ TRAPPIST-1e   ┆ 39.0 ┆ … ┆ 0.0          ┆ 0.0    ┆ 0.0    ┆ false       │
│ Earth      ┆ 0.0       ┆ TRAPPIST-1e   ┆ 24.0 ┆ … ┆ 25.0         ┆ 549.0  ┆ 44.0   ┆ true        │
│ Europa     ┆ 0.0       ┆ TRAPPIST-1e   ┆ 58.0 ┆ … ┆ 0.0          ┆ 6715.0 ┆ 49.0   ┆ false       │
│ Europa     ┆ 0.0       ┆ TRAPPIST-1e   ┆ 33.0 ┆ … ┆ 371.0        ┆ 3329.0 ┆ 193.0  ┆ false       │
│ Earth 

In [25]:
# Same for the Test data

In [26]:
for col in num_cols:
    median_value = train[col].median()  # Use median from training data
    test = test.with_columns(
        pl.when(pl.col(col).is_null())
        .then(median_value)
        .otherwise(pl.col(col))
        .alias(col)
    )

# 2. Encode categorical columns using training encoders
test_cat_df = test.select(cat_cols).to_pandas()

for col in cat_cols:
    encoder = label_encoders[col]  # Use the encoder from the training data
    test_cat_df[col] = test_cat_df[col].fillna("missing")  # Temporarily fill NaN with placeholder
    test_cat_df[col] = encoder.transform(test_cat_df[col])

# Replace placeholder with NaN for imputation
test_cat_df.replace({"missing": np.nan}, inplace=True)

# 3. Impute missing values using the trained KNN Imputer
test_cat_imputed = knn_imputer.transform(test_cat_df)

# Convert imputed values back to original categorical form
for i, col in enumerate(cat_cols):
    test_cat_df[col] = label_encoders[col].inverse_transform(test_cat_imputed[:, i].round().astype(int))

# Convert the imputed data back to a Polars DataFrame
test_cat_imputed_df = pl.DataFrame(test_cat_df)

# Update the original Polars DataFrame with imputed categorical values
test = test.with_columns(test_cat_imputed_df)

print(test)

shape: (4_277, 10)
┌────────────┬───────────┬───────────────┬──────┬───┬───────────┬──────────────┬────────┬────────┐
│ HomePlanet ┆ CryoSleep ┆ Destination   ┆ Age  ┆ … ┆ FoodCourt ┆ ShoppingMall ┆ Spa    ┆ VRDeck │
│ ---        ┆ ---       ┆ ---           ┆ ---  ┆   ┆ ---       ┆ ---          ┆ ---    ┆ ---    │
│ str        ┆ f64       ┆ str           ┆ f64  ┆   ┆ f64       ┆ f64          ┆ f64    ┆ f64    │
╞════════════╪═══════════╪═══════════════╪══════╪═══╪═══════════╪══════════════╪════════╪════════╡
│ Earth      ┆ 1.0       ┆ TRAPPIST-1e   ┆ 27.0 ┆ … ┆ 0.0       ┆ 0.0          ┆ 0.0    ┆ 0.0    │
│ Earth      ┆ 0.0       ┆ TRAPPIST-1e   ┆ 19.0 ┆ … ┆ 9.0       ┆ 0.0          ┆ 2823.0 ┆ 0.0    │
│ Europa     ┆ 1.0       ┆ 55 Cancri e   ┆ 31.0 ┆ … ┆ 0.0       ┆ 0.0          ┆ 0.0    ┆ 0.0    │
│ Europa     ┆ 0.0       ┆ TRAPPIST-1e   ┆ 38.0 ┆ … ┆ 6652.0    ┆ 0.0          ┆ 181.0  ┆ 585.0  │
│ Earth      ┆ 0.0       ┆ TRAPPIST-1e   ┆ 20.0 ┆ … ┆ 0.0       ┆ 635.0        ┆ 0.0    ┆ 

In [27]:
test.describe()

statistic,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
str,str,f64,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""4277""",4277.0,"""4277""",4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0
"""null_count""","""0""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,0.361001,null,28.622866,0.017302,215.062427,428.592238,173.233107,295.895955,304.898293
"""std""",null,0.480347,null,14.029425,0.130409,601.914503,1510.155974,554.991776,1104.872018,1235.991811
"""min""","""Earth""",0.0,"""55 Cancri e""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,0.0,null,20.0,0.0,0.0,0.0,0.0,0.0,0.0
"""50%""",null,0.0,null,27.0,0.0,0.0,0.0,0.0,0.0,0.0
"""75%""",null,1.0,null,37.0,0.0,48.0,66.0,27.0,43.0,31.0
"""max""","""missing""",1.0,"""missing""",79.0,1.0,11567.0,25273.0,8292.0,19844.0,22272.0


# Encoding

In [28]:
# 1. Apply one-hot encoding to categorical columns
one_hot_encoded = train.select(cat_cols).to_dummies()

# 2. Drop original categorical columns from the DataFrame
train = train.drop(cat_cols)  # Pass the list of column names directly

# 3. Concatenate the one-hot encoded columns to the original DataFrame
train = train.hstack(one_hot_encoded)

print(train)


shape: (8_693, 17)
┌───────────┬──────┬─────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
│ CryoSleep ┆ Age  ┆ VIP ┆ RoomService ┆ … ┆ Destination ┆ Destination ┆ Destination ┆ Destination │
│ ---       ┆ ---  ┆ --- ┆ ---         ┆   ┆ _55 Cancri  ┆ _PSO        ┆ _TRAPPIST-1 ┆ _missing    │
│ f64       ┆ f64  ┆ f64 ┆ f64         ┆   ┆ e           ┆ J318.5-22   ┆ e           ┆ ---         │
│           ┆      ┆     ┆             ┆   ┆ ---         ┆ ---         ┆ ---         ┆ u8          │
│           ┆      ┆     ┆             ┆   ┆ u8          ┆ u8          ┆ u8          ┆             │
╞═══════════╪══════╪═════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 0.0       ┆ 39.0 ┆ 0.0 ┆ 0.0         ┆ … ┆ 0           ┆ 0           ┆ 1           ┆ 0           │
│ 0.0       ┆ 24.0 ┆ 0.0 ┆ 109.0       ┆ … ┆ 0           ┆ 0           ┆ 1           ┆ 0           │
│ 0.0       ┆ 58.0 ┆ 1.0 ┆ 43.0        ┆ … ┆ 0           ┆ 0           ┆

In [29]:
# 1. Apply one-hot encoding to categorical columns
one_hot_encoded = test.select(cat_cols).to_dummies()

# 2. Drop original categorical columns from the DataFrame
test = test.drop(cat_cols)  # Pass the list of column names directly

# 3. Concatenate the one-hot encoded columns to the original DataFrame
test = test.hstack(one_hot_encoded)

print(train)


shape: (8_693, 17)
┌───────────┬──────┬─────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
│ CryoSleep ┆ Age  ┆ VIP ┆ RoomService ┆ … ┆ Destination ┆ Destination ┆ Destination ┆ Destination │
│ ---       ┆ ---  ┆ --- ┆ ---         ┆   ┆ _55 Cancri  ┆ _PSO        ┆ _TRAPPIST-1 ┆ _missing    │
│ f64       ┆ f64  ┆ f64 ┆ f64         ┆   ┆ e           ┆ J318.5-22   ┆ e           ┆ ---         │
│           ┆      ┆     ┆             ┆   ┆ ---         ┆ ---         ┆ ---         ┆ u8          │
│           ┆      ┆     ┆             ┆   ┆ u8          ┆ u8          ┆ u8          ┆             │
╞═══════════╪══════╪═════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 0.0       ┆ 39.0 ┆ 0.0 ┆ 0.0         ┆ … ┆ 0           ┆ 0           ┆ 1           ┆ 0           │
│ 0.0       ┆ 24.0 ┆ 0.0 ┆ 109.0       ┆ … ┆ 0           ┆ 0           ┆ 1           ┆ 0           │
│ 0.0       ┆ 58.0 ┆ 1.0 ┆ 43.0        ┆ … ┆ 0           ┆ 0           ┆

In [30]:
train.head(2)

CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_missing,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_missing
f64,f64,f64,f64,f64,f64,f64,f64,bool,u8,u8,u8,u8,u8,u8,u8,u8
0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,false,0,1,0,0,0,0,1,0
0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,true,1,0,0,0,0,0,1,0


In [31]:
test.head(2)

CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_missing,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_missing
f64,f64,f64,f64,f64,f64,f64,f64,u8,u8,u8,u8,u8,u8,u8,u8
1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0
0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,1,0,0,0,0,0,1,0


# Data Modelling

In [32]:
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6oiwrqph
  JVM stdout: /tmp/tmp6oiwrqph/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6oiwrqph/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 19 days
H2O_cluster_name:,H2O_from_python_unknownUser_6o6d8l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [33]:
# Convert Polars DataFrame to Pandas (H2O does not support Polars directly)
train_df = train.to_pandas()
test_df = test.to_pandas()

# Upload Pandas DataFrame to H2O
train_h2o = H2OFrame(train_df)
test_h2o = H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [34]:
# Specify Features and Target
target = "Transported"  # Replace with the name of your target column
features = [col for col in train_h2o.columns if col != target] 

In [35]:
# Train the Model using H2O AutoML
aml = H2OAutoML(max_models=20, seed=42, balance_classes=True)
aml.train(x=features, y=target, training_frame=train_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),1/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [36]:
lb = aml.leaderboard
print(lb)

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20241117_172858  0.880344   0.427076  0.890063                0.206697  0.373705  0.139656
StackedEnsemble_AllModels_1_AutoML_1_20241117_172858     0.880053   0.42732   0.889714                0.203704  0.373917  0.139814
GBM_1_AutoML_1_20241117_172858                           0.878109   0.43264   0.88766                 0.205875  0.375706  0.141155
GBM_5_AutoML_1_20241117_172858                           0.877285   0.435371  0.886985                0.207979  0.376752  0.141942
XGBoost_3_AutoML_1_20241117_172858                       0.87672    0.435788  0.88581                 0.213666  0.377199  0.142279
GBM_2_AutoML_1_20241117_172858                           0.876548   0.435827  0.886332                0.204812  0.3768    0.141978
GBM_grid_1_AutoML_1_20241117_172858_model_1              0.876097   0.436923  0.886

In [53]:
# Predict on Test Data
predictions = aml.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [54]:
predictions_df = predictions.as_data_frame() 
predictions_df.head()

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict,False,True
0,True,0.368184,0.631816
1,False,0.988868,0.011132
2,True,0.011300,0.988700
3,True,0.056430,0.943570
4,True,0.459299,0.540701


# Submission

In [55]:
predictions_df.rename(columns={'predict': 'Transported'}, inplace=True)

In [58]:
import pandas as pd
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [59]:
test_data['PassengerId']

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [60]:
test_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [61]:
submission_df = pd.DataFrame({'PassengerId': test_data['PassengerId'],'Transported': predictions_df['Transported']})

In [62]:
# Save the combined DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [63]:
check = pd.read_csv('submission.csv')

In [64]:
check

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
